Data Ingestion 

In [1]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


In [8]:
%%writefile file.yaml
file_type: txt
dataset_name: TestImages
file_name: jigsaw-toxic-comment-train
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - id
    - comment_text
    - toxic
    - severe_toxic
    - obscene
    - threat
    - insult
    - identity_hate

Overwriting file.yaml


In [9]:
# Read config file
import testutility as util

config_data = util.read_config_file("file.yaml")

In [10]:
config_data['inbound_delimiter']

','

In [11]:
#inspecting data of config file
config_data

{'file_type': 'txt',
 'dataset_name': 'TestImages',
 'file_name': 'jigsaw-toxic-comment-train',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['id',
  'comment_text',
  'toxic',
  'severe_toxic',
  'obscene',
  'threat',
  'insult',
  'identity_hate']}

In [12]:
import pandas as pd
import dask.dataframe as dd
# read the file using config file
file_type = config_data['file_type'] 

dfs = []

source_file = "/content/sample_data/dataset/" + config_data['file_name'] + f'.{file_type}'
# print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
print(df.info())

<ipython-input-12-3b3fd6482849>:10: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
util.col_header_val(df, config_data)

column name and column length validation passed


1

In [ ]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine